In [ ]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from himalaya.backend import set_backend
from matplotlib import pyplot as plt

from compare_variance_residual.residual import residual_method
from compare_variance_residual.simulation import generate_dataset
from compare_variance_residual.variance_partitioning import variance_partitioning

In [ ]:
def get_path(alphas, cv, n_targets):
    path = os.path.join("results", f"targets={n_targets}", f"cv={cv}",
                        f"alphas={alphas.min()},{alphas.max()},{len(alphas)}", "varying dimensions")
    os.makedirs(path, exist_ok=True)
    return path

In [ ]:
def save_scores(d_list_list, scalars, n_targets, n_samples, noise_target, cv, alphas):
    path = get_path(alphas, cv, n_targets)
    for d_list in d_list_list:
        print(d_list)
        csv_path = os.path.join(path, f"scores_{d_list}.csv")
        scores = pd.DataFrame()
        if os.path.exists(csv_path):
            print("skipping, already exists")
            continue
        Xs, Y = generate_dataset(d_list, scalars, n_targets, n_samples, noise_target)
        print("data generated")
        x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score = variance_partitioning(
            Xs, Y, n_samples_train, alphas, cv)
        print("variance partitioning done")

        scores["x1_score"] = x1_score
        scores["x2_score"] = x2_score
        scores["vp_joint_score"] = joint_score
        scores["vp_shared_score"] = x1_and_x2_score
        scores["vp_x1_unique_score"] = vp_x1_unique_score
        scores["vp_x2_unique_score"] = vp_x2_unique_score
        del x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score

        _, _, x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score = residual_method(
            Xs, Y, n_samples_train, alphas, cv)
        print("residual method done")
        scores["rm_x2_to_x1_score"] = np.concatenate(
            [x2_to_x1_score, np.full(len(rm_x1_unique_score) - len(x2_to_x1_score), np.nan)])
        scores["rm_x1_to_x2_score"] = np.concatenate(
            [x1_to_x2_score, np.full(len(rm_x1_unique_score) - len(x1_to_x2_score), np.nan)])
        scores["rm_x1_unique_score"] = rm_x1_unique_score
        scores["rm_x2_unique_score"] = rm_x2_unique_score
        del x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score
        del Xs, Y
        scores.to_csv(csv_path, index=False)

# Save scores for varying Dimensions

In [ ]:
backend = set_backend("cupy", on_error="warn")
warnings.filterwarnings("ignore")
random.seed(42)

In [ ]:
d_list = [100, 100, 100]
n_targets = 10000
n_samples_train = 10000
n_samples_test = 100
n_samples = n_samples_train + n_samples_test
noise_target = 0.1
scalars = [1 / 3, 1 / 3, 1 / 3]

cv = 10
alphas = np.logspace(-5, 5, 10)

In [ ]:
varying_dim = np.logspace(1, 3, 10, dtype=int)

## Shared dimension

In [ ]:
d_list_list = [[int(dim), d_list[1], d_list[2]] for dim in varying_dim]

In [ ]:
save_scores(d_list_list, scalars, n_targets, n_samples, noise_target, cv, alphas)

## Unique dimension

In [ ]:
d_list_list = [[d_list[0], int(dim), d_list[2]] for dim in varying_dim]

In [ ]:
save_scores(d_list_list, d_list, n_targets, n_samples, noise_target, cv, alphas)

# Plot scores

## Shared dimension

In [ ]:
vp = pd.DataFrame()
rm = pd.DataFrame()

for i, dim in enumerate(varying_dim):
    shared_dims = [dim, d_list[1], d_list[2]]
    scores = pd.read_csv(os.path.join(get_path(alphas, cv, n_targets), f"scores_{shared_dims}.csv"))

    vp_scores = pd.DataFrame({
        'shared_dim': dim,
        'vp_x1_unique_score': scores['vp_x1_unique_score'],
    }, index=[i])

    rm_scores = pd.DataFrame({
        'shared_dim': dim,
        'rm_x1_unique_score': scores['rm_x1_unique_score'],
    }, index=[i])

    vp = pd.concat([vp, vp_scores], ignore_index=True)
    rm = pd.concat([rm, rm_scores], ignore_index=True)
vp.head()
rm.head()

In [ ]:
sns.lineplot(data=vp, x='shared_dim', y='vp_x1_unique_score', label='Variance Partitioning', palette='C2')
plt.fill_between(vp['shared_dim'], vp['vp_x1_lower_se'], vp['vp_x1_upper_se'], alpha=0.3)
sns.lineplot(data=rm, x='shared_dim', y='rm_x1_unique_score', label='Residual Method')
plt.fill_between(rm['shared_dim'], rm['rm_x1_lower_se'], rm['rm_x1_upper_se'], alpha=0.3)
plt.axhline(scalars[1], linestyle='--', label='true unique variance')
plt.xlabel(r"$d_\mathbf{A}$")
plt.ylabel(r"Predicted Unique Variance (avg. $R^2$)")
plt.xscale('log')

## Unique dimension

In [ ]:
vp = pd.DataFrame()
rm = pd.DataFrame()

for i, dim in enumerate(varying_dim):
    unique_dims = [d_list[0], dim, d_list[2]]
    scores_unique = pd.read_csv(os.path.join(get_path(alphas, cv, n_targets), f"scores_{unique_dims}.csv"))

    vp_scores = pd.DataFrame({
        'unique_dim': dim,
        'vp_x1_unique_score': scores_unique['vp_x1_unique_score'],
    }, index=[i])

    rm_scores = pd.DataFrame({
        'unique_dim': dim,
        'rm_x1_unique_score': scores_unique['rm_x1_unique_score'],
    }, index=[i])

    vp = pd.concat([vp, vp_scores], ignore_index=True)
    rm = pd.concat([rm, rm_scores], ignore_index=True)

In [ ]:
sns.lineplot(data=vp, x='unique_dim', y='vp_x1_unique_score', label='Variance Partitioning')
plt.fill_between(vp['unique_dim'], vp['vp_x1_lower_se'], vp['vp_x1_upper_se'], alpha=0.3)
sns.lineplot(data=rm, x='unique_dim', y='rm_x1_unique_score', label='Residual Method')
plt.fill_between(rm['unique_dim'], rm['rm_x1_lower_se'], rm['rm_x1_upper_se'], alpha=0.3)
plt.axhline(scalars[1], linestyle='--', label='true unique variance')
plt.xlabel(r"$d_\mathbf{B}$")
plt.ylabel(r"Predicted Unique Variance (avg. $R^2$)")
plt.xscale('log')